In [1]:
#!jupyter nbconvert --to=python load_interp_MLD_python3.ipynb

In [ ]:
class MLDdata:
    '''
    Load MLD Atlas Matlab(c) file and interpolate to FESOM mesh.
    
    Careful: Output still needs to be masked with an ocean bathymetry mask, e.g. with FESOM output!
    
    In: matfileMLD,matvariable,meshpath
    
    Out: class MLDdata with self.mld_int containing interpolated MLD variable
    '''
    
    def __init__(self,runname,resultpath,mesh,matfileMLD,verbose=False):
    
        self.runid = runname
        self.resultpath = resultpath
        self.mesh = mesh
        self.matfileMLD = matfileMLD
        self.verbose = verbose

        import matplotlib.pyplot as plt
        import numpy as np
        from netCDF4 import Dataset
        from scipy.interpolate import griddata
        #import skill_metrics as sm
        #import cartopy.crs as ccrs
        #import pickle

        import pyfesom2 as pf
        from Py_f2recom_toolbox import load_mat

        # load NetCDF ------------------------------------------------------------------------------------
        print('***\nLoading MLD file: {0}\n***'.format(self.matfileMLD))
        
        matMLD = load_mat(self.matfileMLD)
        MLDobs = matMLD['MLClimato']['ML_depth']
        lon = matMLD['MLClimato']['lon']
        lon[lon>180]=lon[lon>180]-360
        lat = matMLD['MLClimato']['lat']
        londic, latdic = np.meshgrid(lon, lat)
        MLDsept = np.squeeze(MLDobs[8,:,:])
        MLDmarc = np.squeeze(MLDobs[2,:,:])
        
#         MLDsept = np.ma.filled(MLDsept, np.nan)
#         MLDmarc = np.ma.filled(MLDmarc, np.nan)
#         MLDsept = np.ma.filled(MLDsept, 0)
#         MLDmarc = np.ma.filled(MLDmarc, 0)
        MLDmarc[MLDmarc ==0] = np.nan
        MLDsept[MLDsept ==0] = np.nan
        
        # load FESOM mesh diag -------------------------------------------------------------------------------
        meshdiag= self.mesh.path+'/'+self.runid+'.mesh.diag.nc'
        #!ncdump -h $meshdiag

        diag = pf.get_meshdiag(mesh,meshdiag=meshdiag, runid=self.runid)
        if 'nl' in diag.dims.mapping.keys():
            #nod_area = diag.rename_dims({"nl": "nz1", "nod_n": "nod2"}).nod_area
            #nod_area.load()
            mesh_depths = diag['Z'].values
        elif 'nz1' in diag.dims.mapping.keys():
            mesh_depths = diag['nz1'].values

        # storage container
        mld_marc_int = np.zeros(len(mesh.x2))
        mld_sept_int = np.zeros(len(mesh.x2))
        
        aux1 = np.zeros((MLDmarc.size))*np.nan
        aux2 = np.zeros((MLDmarc.size))*np.nan

        ind1   = np.squeeze(~np.isnan(MLDmarc)) 
        ind2   = np.squeeze(~np.isnan(MLDsept)) 

        # first interpolation to original grid to close empty parts
        aux1 = griddata((londic[ind1], latdic[ind1]), MLDmarc[ind1], (londic, latdic), method='nearest')                             
        aux2 = griddata((londic[ind2], latdic[ind2]), MLDsept[ind2], (londic, latdic), method='nearest')                             
        # 2D field without nans                           
        
        aux1[~ind1] = np.nan
        aux2[~ind2] = np.nan
        
        # second interpolation to FESOM grid
        mld_marc_int[:] = griddata((londic.ravel(), latdic.ravel()), aux1.ravel(), 
                                   (mesh.x2, mesh.y2), method='nearest')
        mld_sept_int[:] = griddata((londic.ravel(), latdic.ravel()), aux2.ravel(), 
                                   (mesh.x2, mesh.y2), method='nearest')  
        # Final interpolated field

        if np.isnan(np.min(mld_marc_int)): print('WARNING: The interpolated field contains NaNs')  # Testing if results contain NaNs. If yes, the routine needs adjustments

        if(self.verbose):
            print('Winter MLD: min = {0} max = {1} mean = {2}'.format(np.nanmin(mld_marc_int), np.nanmax(mld_marc_int), np.nanmean(mld_marc_int)))
            print('Summer MLD: min = {0} max = {1} mean = {2}'.format(np.nanmin(mld_sept_int), np.nanmax(mld_sept_int), np.nanmean(mld_sept_int)))

        #mld_sept_int = np.swapaxes(mld_sept_int,0,1) # adjust axes layout to FESOM output
        #mld_marc_int = np.swapaxes(mld_marc_int,0,1)
        #print(np.shape(woa_int))    
        
        self.layer_depths = mesh_depths
        self.mld_sept_int = mld_sept_int
        self.mld_marc_int = mld_marc_int
        self.lon = lon
        self.lat = lat